In [1]:
import os
import sys
import json
import numpy as np
import pandas as pd
import re
from collections import defaultdict
import json
import subprocess
from collections import Counter
import gc
import time


In [4]:
file_vcf="file1.vcf"

In [ ]:
with open(file_vcf) as vcf_in:
    for line in vcf_in:
        if line[0]=='#':
            continue
        tmp_Variation,tmp_Location,tmp_Allele,tmp_Gene,tmp_Feature,tmp_Feature_type,tmp_Consequence,tmp_cDNA_position,tmp_CDS_position,tmp_Protein_position,tmp_Amino_acids,tmp_Codons,tmp_Existing_variation,tmp_Extra = line.rstrip().split('\t')
        tmp_annot=tmp_Consequence
        if 'SYMBOL=' in tmp_Extra:
            result=re.search('SYMBOL=(.*);SYMBOL_SOURCE', tmp_Extra)
            tmp_gene=result.group(1)
        elif 'SYMBOL=' not in tmp_Extra:
            continue
        #if more than one annotation,should be assign the most deleterious one
        #list_annotation=tmp_annot.split(',')
        if 'MANE' not in tmp_Extra:
            if len(list_annotation) == 1:
                dicts[tmp_Variation][tmp_gene].append(list_annotation[0])
            elif len(list_annotation) > 1:
                for annot_in_list in list_annotation:
                    dicts[tmp_Variation][tmp_gene].append(annot_in_list)
        elif 'MANE' in tmp_Extra:
            if len(list_annotation) == 1:
                dicts[tmp_Variation][tmp_gene].append('MANE_'+list_annotation[0])
            elif len(list_annotation) > 1:
                for annot_in_list in list_annotation:
                    dicts[tmp_Variation][tmp_gene].append('MANE_'+annot_in_list)

In [22]:

# Create an empty list to store extracted data
data = []

with open(file_vcf) as vcf_in:
    for line in vcf_in:
        if line.startswith('#'):
            continue  # Skip header lines
        
        fields = line.rstrip().split('\t')  # Split by tab
        
        # Extract required columns
        uploaded_variation = fields[0]
        consequence = fields[6]
        tmp_Extra = fields[13]
        
        if 'SYMBOL=' in tmp_Extra:
            result=re.search('SYMBOL=(.*);SYMBOL_SOURCE', tmp_Extra)
            tmp_gene=result.group(1)
        elif 'SYMBOL=' not in tmp_Extra:
            continue

        # Extract MANE_SELECT
        if 'MANE_SELECT=' in tmp_Extra:
            #result=re.search('MANE_SELECT=(.*);TSL', tmp_Extra)
            result=re.search(r"MANE_SELECT=([^;]+)", tmp_Extra)
            mane_select=result.group(1)
        elif 'MANE_SELECT=' not in tmp_Extra:
            mane_select="-"
            
        if 'MANE_PLUS_CLINICAL=' in tmp_Extra:
            #result=re.search('MANE_PLUS_CLINICAL=(.*);CCDS', tmp_Extra)
            result=re.search(r"MANE_PLUS_CLINICAL=([^;]+)", tmp_Extra)
            mane_plus_clinical=result.group(1)
        elif 'MANE_PLUS_CLINICAL=' not in tmp_Extra:
            mane_plus_clinical="-"
        
        # Append extracted data to the list
        data.append([uploaded_variation, consequence, tmp_gene, mane_select, mane_plus_clinical])

# Create a DataFrame
df = pd.DataFrame(data, columns=['Uploaded_variation', 'Consequence', 'SYMBOL', 'MANE_SELECT', 'MANE_PLUS_CLINICAL'])

# Print the first few rows
display(df)



,Uploaded_variation,Consequence,SYMBOL,MANE_SELECT,MANE_PLUS_CLINICAL
0,1:16666:G:A,downstream_gene_variant,DDX11L1,-,-
1,1:16666:G:A,downstream_gene_variant,DDX11L2,-,-
2,1:16666:G:A,non_coding_transcript_exon_variant,WASH7P,-,-
3,1:16666:G:A,downstream_gene_variant,MIR6859-1,-,-
4,1:16792:G:A,downstream_gene_variant,DDX11L1,-,-
...,...,...,...,...,...
3714806,X:156025488:G:A,non_coding_transcript_exon_variant,WASH6P,-,-
3714807,X:156025488:G:A,downstream_gene_variant,WASH6P,-,-
3714808,X:156025488:G:A,non_coding_transcript_exon_variant,WASH6P,-,-
3714809,X:156025488:G:A,downstream_gene_variant,DDX11L16,-,-


In [23]:
df.to_csv('myVCF.trial.tsv', sep='\t', index=False)

In [24]:
display(df[(df['MANE_SELECT'] != '-') | (df['MANE_PLUS_CLINICAL'] != '-')])


,Uploaded_variation,Consequence,SYMBOL,MANE_SELECT,MANE_PLUS_CLINICAL
223,1:925761:C:T,intron_variant,SAMD11,NM_001385641.1,-
234,1:925881:G:A,intron_variant,SAMD11,NM_001385641.1,-
245,1:925912:C:G,"splice_polypyrimidine_tract_variant,intron_var...",SAMD11,NM_001385641.1,-
255,1:926137:C:T,intron_variant,SAMD11,NM_001385641.1,-
265,1:930055:C:G,intron_variant,SAMD11,NM_001385641.1,-
...,...,...,...,...,...
3713989,X:156007429:G:A,intron_variant,IL9R,NM_002186.3,-
3713993,X:156007526:G:A,synonymous_variant,IL9R,NM_002186.3,-
3713997,X:156007547:C:T,synonymous_variant,IL9R,NM_002186.3,-
3714001,X:156009974:G:A,synonymous_variant,IL9R,NM_002186.3,-


In [ ]:
 df = pd.read_csv(file_bed, sep='\t', comment='#')
    df.columns=['Uploaded_variation', 'SYMBOL', 'Consequence', 'MANE_SELECT', 'MANE_PLUS_CLINICAL']
    df = pd.read_csv(file_bed, sep='\t', comment='#')
    df.columns=['Uploaded_variation', 'SYMBOL', 'Consequence', 'MANE_SELECT', 'MANE_PLUS_CLINICAL']